# Setup

In [1]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [2]:
%load_ext autoreload
%autoreload 2

from scripts.api import *

from scripts.create_graph import *
import networkx as nx
import networkx.algorithms.community as nx_comm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime

pd.set_option("display.max_colwidth", None)

# Read Data - 5 gpt generations data

In [25]:
# get column name
col_name = ['created_at', 'screen_name', 'text', 'lang', 'retweet_count',
       'reply_count', 'like_count', 'quote_count', 'id', 'author_id',
       'conversation_id', 'in_reply_to_user_id', 'geo', 'entities']

In [8]:
# read before data
fname_db = f"data/esther"  
before_1 = DB.fetch(table_name='before_1', path=fname_db)
before_1.columns = col_name
before_2 = DB.fetch(table_name='before_2', path=fname_db)
before_2.columns = col_name
before_3 = DB.fetch(table_name='before_3', path=fname_db)
before_3.columns = col_name
before_35 = DB.fetch(table_name='before_35', path=fname_db)
before_35.columns = col_name
before_4 = DB.fetch(table_name='before_4', path=fname_db)
before_4.columns = col_name

In [12]:
# read after data
after_1 = DB.fetch(table_name='after_1', path=fname_db)
after_1.columns = col_name
after_2 = DB.fetch(table_name='after_2', path=fname_db)
after_2.columns = col_name
after_3 = DB.fetch(table_name='after_3', path=fname_db)
after_3.columns = col_name
after_35 = DB.fetch(table_name='after_35', path=fname_db)
after_35.columns = col_name
after_4 = DB.fetch(table_name='after_4', path=fname_db)
after_4.columns = col_name

# Detect Sentiment

In [15]:
import umap
import scripts.TextAnalysis as ta
from scripts.api import *
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import codecs  #this let's us display tweets properly (emojis, etc.)

import sklearn.cluster as cluster
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy import stats

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [16]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

ImportError: 
AutoModelForSequenceClassification requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFAutoModelForSequenceClassification".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.


In [20]:
def sentiment_classifier(text,model,tokenizer):
    inputs = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True)

    token_type_ids = inputs['token_type_ids']
    input_ids = inputs['input_ids']

    output = model(input_ids, token_type_ids=token_type_ids,return_dict=True,output_hidden_states=True)
    logits = np.array(output.logits.tolist()[0])
    prob = np.exp(logits)/np.sum(np.exp(logits))
    sentiment = np.sum([(x+1)*prob[x] for x in range(len(prob))])  #use this line if you want the mean score
    
    return sentiment